# Introduction

Llama 3.1 is currently (as of Aug 2024) the latest model from Meta. Let's try and see what this model is bringing.

Will test now the following model:

* **Model**: Llama3.1
* **Version**: 8b-instruct
* **Framework**: Transformers
* **Version**: V1

This is what we will test:

* Simple prompts with general information questions
* Poetry (haiku, sonets) writing
* Code writing (Python, C++, Java)
* Software design (simple problems)
* Multi-parameter questions
* Chain of reasoning
* A more complex reasoning problem


**Note**: this notebook is organized to facilitate a comparison of **Llama3.1** with **Gemma 2**. Corresponding notebook is [Unlock the Power of Gemma 2: Prompt it like a Pro](https://www.kaggle.com/code/gpreda/unlock-the-power-of-gemma-2-prompt-it-like-a-pro). At the end of current notebook we are reviewing the performance of the two notebooks for the list of tasks submitted to both.

# Preparation

We import the libraries we need, and we set the model to be ready for testing.

## Import packages

In [1]:
%%capture
%pip install -U transformers accelerate

In [2]:
from time import time
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
from IPython.display import display, Markdown

2024-08-11 16:26:52.712968: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-11 16:26:52.713081: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-11 16:26:52.851326: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Define model

This step might take a bit more time, around 2 minutes.

In [3]:
model_id = "/kaggle/input/llama-3.1/transformers/8b-instruct/1"

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
        model_id,
        return_dict=True,
        low_cpu_mem_usage=True,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True,
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
llama31_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)

## Prepare query function

In [5]:
def query_model(
        system_message,
        user_message,
        temperature=0.7,
        max_length=1024
        ):
    start_time = time()
    user_message = "Question: " + user_message + " Answer:"
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
        ]
    prompt = llama31_pipeline.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
        )
    terminators = [
        llama31_pipeline.tokenizer.eos_token_id,
        llama31_pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
    sequences = llama31_pipeline(
        prompt,
        do_sample=True,
        top_p=0.9,
        temperature=temperature,
        num_return_sequences=1,
        eos_token_id=terminators,
        max_new_tokens=max_length,
        return_full_text=False,
        pad_token_id=terminators[0]
    )
    #answer = f"{sequences[0]['generated_text'][len(prompt):]}\n"
    answer = sequences[0]['generated_text']
    end_time = time()
    ttime = f"Total time: {round(end_time-start_time, 2)} sec."

    return user_message + " " + answer  + " " +  ttime


system_message = """
You are an AI assistant designed to answer simple questions.
Please restrict your answer to the exact question asked.
"""

## Utility function for formatting the output


In [6]:
def colorize_text(text):
    for word, color in zip(["Reasoning", "Question", "Answer", "Total time"], ["blue", "red", "green", "magenta"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

# Test the model


## Test with few simple geography and history questions

In [7]:
t1 = time()
response = query_model(
    system_message,
    user_message="What is the surface temperature of the Moon?",
    temperature=0.1,
    max_length=256)
display(Markdown(colorize_text(f"{response}")))



**<font color='red'>Question:</font>** What is the surface temperature of the Moon? 

**<font color='green'>Answer:</font>** The surface temperature of the Moon varies greatly between day and night. During the lunar day, which lasts about 29.5 Earth days, the surface temperature can reach as high as 127°C (261°F) in the direct sunlight. In contrast, during the lunar night, the temperature can drop to as low as -173°C (-279°F) in the permanently shadowed craters near the lunar poles. 

**<font color='magenta'>Total time:</font>** 52.53 sec.

Let's ask a more simple question, from European geography.

In [8]:
response = query_model(    
    system_message,
    user_message="What is the surface of France?",
    temperature=0.1,
    max_length=128)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** What is the surface of France? 

**<font color='green'>Answer:</font>** The surface of France is approximately 643,801 square kilometers. 

**<font color='magenta'>Total time:</font>** 8.75 sec.

Now, let's continue with some realy tough questions, from Ancient Greece, medieval Japan and more.

In [9]:
response = query_model(
    system_message,
    user_message="When was the 30 years war?",
    temperature=0.1, 
    max_length=128)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** When was the 30 years war? 

**<font color='green'>Answer:</font>** The 30 Years War was a complex conflict that lasted from 1618 to 1648. 

**<font color='magenta'>Total time:</font>** 13.14 sec.

In [10]:
response = query_model(
    system_message,
    user_message="What is graphe paranomon?",
    temperature=0.1, 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** What is graphe paranomon? 

**<font color='green'>Answer:</font>** Graphe paranomon is a term used in ancient Greek law. It refers to a type of impeachment or indictment that was brought against a magistrate or public official in ancient Athens. 

**<font color='magenta'>Total time:</font>** 22.88 sec.

In [11]:
response = query_model(
    system_message,
    user_message="Who was the next shogun after Tokugawa Ieyasu?",
    temperature=0.1, 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Who was the next shogun after Tokugawa Ieyasu? 

**<font color='green'>Answer:</font>** Tokugawa Hidetada. 

**<font color='magenta'>Total time:</font>** 5.67 sec.

In [12]:
response = query_model(
    system_message,
    user_message="What was the name of the Chinese dinasty during 1st century B.C.?",
    temperature=0.1, 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** What was the name of the Chinese dinasty during 1st century B.C.? 

**<font color='green'>Answer:</font>** The Han Dynasty. 

**<font color='magenta'>Total time:</font>** 3.21 sec.

Let's try now some questions from American history. These must be simpler.

In [13]:
response = query_model(
    system_message,
    user_message="Who was the first American president?",
    temperature=0.1, 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Who was the first American president? 

**<font color='green'>Answer:</font>** George Washington. 

**<font color='magenta'>Total time:</font>** 2.57 sec.

In [14]:
response = query_model(
    system_message,
    user_message="When took place the Civil War in United States of America?",
    temperature=0.1, 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** When took place the Civil War in United States of America? 

**<font color='green'>Answer:</font>** The American Civil War took place from 1861 to 1865. 

**<font color='magenta'>Total time:</font>** 9.93 sec.

## Let's write poetry now

In [15]:
system_message = """
You are an AI assistant designed to write poetry.
Please answer with a haiku format (17 words poems).
Question: {question}
Answer:
"""
response = query_model(
    system_message,
    user_message="Please write a poem about Boris Becker wins in tennis",
    temperature=0.1, 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Please write a poem about Boris Becker wins in tennis 

**<font color='green'>Answer:</font>** Racket's mighty swing falls
Golden ace on Wimbledon's grass
Victory's sweet cry 

**<font color='magenta'>Total time:</font>** 12.42 sec.

In [16]:
response = query_model(
    system_message,
    user_message="Please write a poem about Shakespeare being lame at playing poker",
    temperature=0.1, 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Please write a poem about Shakespeare being lame at playing poker 

**<font color='green'>Answer:</font>** Bard's bluffing fails miserably
Shakespeare's poker face a joke
Lame, his royal flush 

**<font color='magenta'>Total time:</font>** 14.83 sec.

In [17]:
system_message = """
You are an AI assistant designed to write poetry.
Please answer with a short poem, with rime, in the style of Shakespeare's poems.
Question: {question}
Answer:
"""
response = query_model(
    system_message,
    user_message="Please write a poem about Nadia Comaneci winning Montreal Olympiad",
    temperature=0.1, 
    max_length=512)
display(Markdown(colorize_text(response)))

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset




**<font color='red'>Question:</font>** Please write a poem about Nadia Comaneci winning Montreal Olympiad 

**<font color='green'>Answer:</font>** O, fairest Nadia, thou didst shine so bright,
At Montreal's games, where glory doth take flight.
Thy gymnastics skills, a wondrous sight to see,
Didst earn thee perfect tens, a feat most rare to be.

Thy body, lithe and strong, didst dance with ease,
Through routines, a symphony of skill and peace.
The judges' scores, a chorus of delight,
Didst proclaim thee perfect, in that moment's light.

The crowd, in awe, didst cheer and shout thy name,
As thou didst claim the gold, and Olympic fame.
Thy country's pride, didst swell with joy and cheer,
For thou, fair Nadia, hadst brought them near.

To greatness, thou didst rise, like morning sun,
And in the annals of time, thy name is won.
A legend born, of skill and youthful might,
Nadia Comaneci, shining bright, in Olympic light. 

**<font color='magenta'>Total time:</font>** 124.61 sec.

## Math problems and Python code

Let's continue now with some math problems for which the response should be given by Python code.

In [18]:
system_message = """
You are an AI assistant designed to write simple Python code.
Please answer with the listing of the Python code.
Question: {question}
Answer:
"""
response = query_model(
    system_message,
    user_message="Please write a function in Python to calculate the area of a circle of radius r",
    temperature=0.1, 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Please write a function in Python to calculate the area of a circle of radius r 

**<font color='green'>Answer:</font>** ```python
import math

def calculate_circle_area(r):
    """
    Calculate the area of a circle given its radius.

    Args:
        r (float): The radius of the circle.

    Returns:
        float: The area of the circle.
    """
    return math.pi * (r ** 2)

# Example usage:
radius = 5
area = calculate_circle_area(radius)
print(f"The area of the circle with radius {radius} is {area:.2f}")
``` 

**<font color='magenta'>Total time:</font>** 62.49 sec.

In [19]:
response = query_model(
    system_message,
    user_message="Please write a function in Python to order a list",
    temperature=0.1, 
    max_length=256)
display(Markdown(colorize_text(response)))




**<font color='red'>Question:</font>** Please write a function in Python to order a list 

**<font color='green'>Answer:</font>** ```python
def order_list(input_list):
    """
    This function takes a list as input and returns the list in sorted order.
    
    Args:
        input_list (list): The list to be sorted.
    
    Returns:
        list: The sorted list.
    """
    return sorted(input_list)

# Example usage:
numbers = [64, 34, 25, 12, 22, 11, 90]
print(order_list(numbers))  # Output: [11, 12, 22, 25, 34, 64, 90]
``` 

**<font color='magenta'>Total time:</font>** 74.02 sec.

## Software design

In [20]:
response = query_model(
    system_message,
    user_message="""Please write a class in Python 
                        to model a phone book (storing name, surname, address, phone) 
                        with add, delete, order by name, search operations.
                        The class should store a list of contacts, each
                        with name, surname, address, phone information stored.
                        """,
    temperature=0.1, 
    max_length=1024)
display(Markdown(colorize_text(response)))




**<font color='red'>Question:</font>** Please write a class in Python 
                        to model a phone book (storing name, surname, address, phone) 
                        with add, delete, order by name, search operations.
                        The class should store a list of contacts, each
                        with name, surname, address, phone information stored.
                         

**<font color='green'>Answer:</font>** ```python
class Contact:
    def __init__(self, name, surname, address, phone):
        self.name = name
        self.surname = surname
        self.address = address
        self.phone = phone

    def __str__(self):
        return f"{self.name} {self.surname}, {self.address}, {self.phone}"


class PhoneBook:
    def __init__(self):
        self.contacts = []

    def add_contact(self, name, surname, address, phone):
        new_contact = Contact(name, surname, address, phone)
        self.contacts.append(new_contact)

    def delete_contact(self, name, surname):
        for contact in self.contacts:
            if contact.name == name and contact.surname == surname:
                self.contacts.remove(contact)
                print(f"Contact {name} {surname} deleted.")
                return
        print(f"Contact {name} {surname} not found.")

    def order_by_name(self):
        self.contacts.sort(key=lambda x: (x.name, x.surname))

    def search_contact(self, name, surname):
        for contact in self.contacts:
            if contact.name == name and contact.surname == surname:
                return contact
        return None

    def display_contacts(self):
        self.order_by_name()
        for contact in self.contacts:
            print(contact)


# Example usage:
phone_book = PhoneBook()

while True:
    print("\n1. Add contact")
    print("2. Delete contact")
    print("3. Order by name")
    print("4. Search contact")
    print("5. Display contacts")
    print("6. Exit")

    choice = input("Choose an option: ")

    if choice == "1":
        name = input("Enter name: ")
        surname = input("Enter surname: ")
        address = input("Enter address: ")
        phone = input("Enter phone: ")
        phone_book.add_contact(name, surname, address, phone)
    elif choice == "2":
        name = input("Enter name: ")
        surname = input("Enter surname: ")
        phone_book.delete_contact(name, surname)
    elif choice == "3":
        phone_book.order_by_name()
    elif choice == "4":
        name = input("Enter name: ")
        surname = input("Enter surname: ")
        contact = phone_book.search_contact(name, surname)
        if contact:
            print(contact)
        else:
            print("Contact not found.")
    elif choice == "5":
        phone_book.display_contacts()
    elif choice == "6":
        break
    else:
        print("Invalid option.")
``` 

**<font color='magenta'>Total time:</font>** 325.47 sec.

In [21]:
response = query_model(
    system_message,
    user_message="""Please write a small Python module that creates a REST API service
                        with two endpoints: 
                        * get_status (GET)
                        * prediction (POST)
                        The prediction endpoint receives the payload, extract three fields: city, street and number
                        and returns a field called price_estimate
                        
                        """,
    temperature=0.1, 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Please write a small Python module that creates a REST API service
                        with two endpoints: 
                        * get_status (GET)
                        * prediction (POST)
                        The prediction endpoint receives the payload, extract three fields: city, street and number
                        and returns a field called price_estimate
                        
                         

**<font color='green'>Answer:</font>** ```python
from flask import Flask, request, jsonify
from flask_restful import Api, Resource

app = Flask(__name__)
api = Api(app)

class Status(Resource):
    def get(self):
        return {'status': 'ok'}

class Prediction(Resource):
    def post(self):
        data = request.json
        if 'city' not in data or'street' not in data or 'number' not in data:
            return {'error': 'Missing required fields'}, 400
        city = data['city']
        street = data['street']
        number = data['number']
        # For simplicity, let's assume we have a function that estimates the price based on the input
        def estimate_price(city, street, number):
            # This is a placeholder function, you should replace it with your actual logic
            return 1000 + int(number) * 10 + len(city) * 5 + len(street) * 2

        price_estimate = estimate_price(city, street, number)
        return {'price_estimate': price_estimate}

api.add_resource(Status, '/')
api.add_resource(Prediction, '/predict')

if __name__ == '__main__':
    app.run(debug=True)
```

This code creates a simple REST 

**<font color='magenta'>Total time:</font>** 155.78 sec.

## C++ code

We continue now with some C++ code.

In [22]:
system_message = """
You are an AI assistant designed to write simple C++ code.
Please answer with the listing of the C++ code.
Question: {question}
Answer:
"""
response = query_model(
    system_message,
    user_message="Please write a function in C++ to calculate the area of a circle of radius r", 
    temperature=0.1, 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Please write a function in C++ to calculate the area of a circle of radius r 

**<font color='green'>Answer:</font>** ```cpp
#include <iostream>
#include <cmath>

double calculate_circle_area(double r) {
    return M_PI * pow(r, 2);
}

int main() {
    double radius = 5.0;
    std::cout << "The area of the circle with radius " << radius << " is " << calculate_circle_area(radius) << std::endl;
    return 0;
}
``` 

**<font color='magenta'>Total time:</font>** 50.46 sec.

In [23]:
system_message = """
You are an AI assistant designed to write simple C++ code.
Please answer with the listing of the C++ code.
Question: {question}
Answer:
"""
response = query_model(
    system_message,
    user_message="Please write a function in C++ to calculate the volume of a cylinder with radius r and height h",
    temperature=0.1,
    max_length=512)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Please write a function in C++ to calculate the volume of a cylinder with radius r and height h 

**<font color='green'>Answer:</font>** ```cpp
#include <iostream>

double cylinder_volume(double r, double h) {
    return 3.14159 * r * r * h;
}

int main() {
    double r = 5.0;
    double h = 10.0;
    std::cout << "The volume of the cylinder is: " << cylinder_volume(r, h) << std::endl;
    return 0;
}
``` 

**<font color='magenta'>Total time:</font>** 53.54 sec.

In [24]:
response = query_model(
    system_message,
    user_message="Please write a function in C++ to order a list", 
    temperature=0.1, 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Please write a function in C++ to order a list 

**<font color='green'>Answer:</font>** ```cpp
#include <iostream>
#include <vector>
#include <algorithm>

std::vector<int> orderList(std::vector<int> lst) {
    std::sort(lst.begin(), lst.end());
    return lst;
}

int main() {
    std::vector<int> numbers = {64, 34, 25, 12, 22, 11, 90};
    std::vector<int> orderedNumbers = orderList(numbers);
    for (int num : orderedNumbers) {
        std::cout << num << " ";
    }
    return 0;
}
``` 

**<font color='magenta'>Total time:</font>** 72.93 sec.

## Multiple parameters questions

### Best food in France

In [25]:
system_message = """
You are an AI assistant designed to answer questions with parameters.
Return the answer formated nicely, for example with bullet points.
"""
user_message = """
What are the {adjective} {number} {items} from {place}?
"""
response = query_model(
    system_message,
    user_message.format(
    adjective="best",
    number="3",
    items="food",
    place="France"
    ), 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** 
What are the best 3 food from France?
 

**<font color='green'>Answer:</font>** Bonjour! Here are three of the most iconic and delicious foods from France:

* **Escargots (Snails in Garlic Butter)**: A classic French appetizer made from snails cooked in garlic butter, parsley, and white wine. Typically served as an appetizer or starter.
* **Coq au Vin (Chicken in Red Wine)**: A braised chicken dish cooked in red wine, mushrooms, onions, and bacon, resulting in tender and flavorful meat. Often served with boiled potatoes and vegetables.
* **Crème Brûlée (Burnt Cream)**: A rich dessert consisting of creamy custard base topped with a layer of caramelized sugar. The texture and flavors of the creamy custard and caramelized sugar create a delightful contrast.

Bon appétit! 

**<font color='magenta'>Total time:</font>** 96.08 sec.

### Best attractions in Italy

In [26]:
response = query_model(    
    system_message,
    user_message.format(
    adjective="best",
    number="five",
    items="attractions",
    place="Italy"
    ), 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** 
What are the best five attractions from Italy?
 

**<font color='green'>Answer:</font>** Here are the top 5 attractions in Italy:

• **The Colosseum (Rome)**: This ancient amphitheater is one of Rome's most iconic landmarks and a testament to the city's rich history. Built in the 1st century AD, it could hold up to 50,000 spectators and hosted gladiator battles, animal hunts, and public executions.

• **The Leaning Tower of Pisa (Pisa)**: This famous tower is a marvel of medieval engineering and a popular tourist destination. Built in the 12th century, it's known for its unintended tilt, which has become a symbol of the city of Pisa.

• **The Uffizi Gallery (Florence)**: This world-renowned art museum is home to some of the most famous paintings in history, including Botticelli's "The Birth of Venus" and Leonardo da Vinci's "Annunciation." The gallery is located in a 16th-century palace and features a stunning courtyard and ornate decorations.

• **The Grand Canal (Venice)**: This serene waterway is the main thoroughfare of Venice, lined with beautiful palaces, ornate bridges, and picturesque piazzas. Take a romantic gondola ride or stroll along the canal 

**<font color='magenta'>Total time:</font>** 155.67 sec.

### Most affordable places to stay in Spain 

In [27]:
response = query_model(
    system_message,
    user_message.format(
    adjective="most affordable",
    number="two",
    items="places to retire",
    place="Spain"
    ), 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** 
What are the most affordable two places to retire from Spain?
 

**<font color='green'>Answer:</font>** Here are two of the most affordable places to retire from Spain:

**1. Valencia Region**

* **Cost of living index:** 74.25 (compared to the national average of 100)
* **Rent:** €400-€600 per month for a one-bedroom apartment
* **Food:** Meals from €5-€10
* **Transportation:** €1.50 for a single bus ticket
* **Healthcare:** Excellent public healthcare system with low costs
* **Climate:** Mild Mediterranean climate with warm summers and mild winters

The Valencia region is a popular destination for retirees due to its low cost of living, beautiful beaches, and rich cultural heritage. The city of Valencia is a great place to live, with a vibrant atmosphere and a wide range of cultural and recreational activities.

**2. Málaga Province**

* **Cost of living index:** 71.25 (compared to the national average of 100)
* **Rent:** €350-€550 per month for a one-bedroom apartment
* **Food:** Meals from €3-€8
* **Transportation:** €1.20 for a single bus ticket
* **Healthcare:** Excellent public healthcare system with low costs
* **Climate:** Mild Mediterranean climate 

**<font color='magenta'>Total time:</font>** 155.46 sec.

### Less known but great places to stay in Romania

In [28]:
response = query_model(    
    system_message,
    user_message.format(
    adjective="Less known but great",
    number="4",
    items="places to stay",
    place="Romania"
    ), 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** 
What are the Less known but great 4 places to stay from Romania?
 

**<font color='green'>Answer:</font>** Here are some lesser-known but great places to stay in Romania:

* **Maramureș County:**
	+ **Bârsana**: A small village in the heart of the Maramureș region, known for its traditional wooden churches and folk culture. Stay at a traditional guesthouse or a local family-run pension.
	+ **Sighetu Marmației**: A charming town with a rich history and culture, featuring a beautiful synagogue, a museum, and a lively market. Stay at a cozy hotel or a boutique guesthouse.
* **Bistrița-Năsăud County:**
	+ **Bistrița**: A historic city with a rich cultural heritage, featuring a beautiful old town and a stunning castle. Stay at a hotel or a guesthouse in the city center.
	+ **Râșca**: A picturesque village surrounded by the Carpathian Mountains, known for its traditional wooden churches and folk culture. Stay at a guesthouse or a local family-run pension.
* **Dolj County:**
	+ **Craiova**: A charming city with a rich history and culture, featuring a beautiful old town and a stunning castle. Stay at a hotel or a boutique guesthouse.
	 

**<font color='magenta'>Total time:</font>** 155.55 sec.

### Best commedies by Shakespeare


In [29]:
response = query_model(
    system_message,
    user_message.format(
    adjective="best",
    number="3",
    items="commedies",
    place="Shakespeare entire creation"
    ), 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** 
What are the best 3 commedies from Shakespeare entire creation?
 

**<font color='green'>Answer:</font>** A delightful question! While opinions may vary, here are three of the most popular and enduring comedies from Shakespeare's works, widely regarded as among the best:

• **A Midsummer Night's Dream** (1595-96)
	+ A lighthearted and enchanting tale of love, magic, and mischief in the forest, featuring the iconic characters of Puck, Oberon, and the lovestruck young lovers.
	+ Features some of Shakespeare's most beautiful and poetic language, including the famous "What fools these mortals be" soliloquy.

• **Twelfth Night** (1599-1602)
	+ A witty and charming comedy of mistaken identities, love triangles, and disguise, set in the court of Duke Orsino.
	+ Includes some of Shakespeare's most memorable characters, such as the quick-witted Viola, the lovestruck Orsino, and the bumbling Malvolio.

• **As You Like It** (1599-1600)
	+ A pastoral comedy of love, family, and identity, set in the Forest of Arden, where the young Rosalind disguises herself as a man to escape her uncle's tyranny and find her true love.
	 

**<font color='magenta'>Total time:</font>** 155.45 sec.

### Most important battles fromn WW2

In [30]:
response = query_model(
    system_message,
    user_message.format(
    adjective="most important",
    number="5",
    items="battles",
    place="WW2"
    ), 
    max_length=512)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** 
What are the most important 5 battles from WW2?
 

**<font color='green'>Answer:</font>** Here are the top 5 most important battles of World War II:

**Top 5 Most Important Battles of World War II:**

* **1. **The Battle of Stalingrad (August 1942 - February 1943)**:
	+ Location: Stalingrad, Russia
	+ Outcome: Decisive Soviet victory, marked a major turning point on the Eastern Front, and led to the eventual defeat of Nazi Germany
	+ Significance: Largest battle in history, with over 1.8 million casualties, and marked a shift in momentum in favor of the Allies
* **2. **The Battle of Midway (June 4-7, 1942)**:
	+ Location: Midway Atoll, Hawaii
	+ Outcome: Decisive American victory, prevented a potential Japanese invasion of Hawaii and halted Japanese expansion in the Pacific
	+ Significance: Considered a turning point in the Pacific War, as it thwarted Japan's plans for further expansion and set the stage for the Allied counterattack
* **3. **The Battle of Normandy (June 6, 1944 - August 21, 1944)**:
	+ Location: Normandy, France
	+ Outcome: Allied victory, marked the beginning of the end of Germany's control over Western Europe
	+ Significance: Largest seaborne invasion in history, led to the liberation of Western Europe and paved the way for the Allied advance into Germany
* **4. **The Battle of Berlin (April 16 - May 2, 1945)**:
	+ Location: Berlin, Germany
	+ Outcome: Soviet victory, led to the capture of Berlin and the eventual surrender of Germany
	+ Significance: Marked the end of World War II in Europe, and led to the division of Germany and the beginning of the Cold War
* **5. **The Battle of El Alamein (October 23 - November 11, 1942)**:
	+ Location: El Alamein, Egypt
	+ Outcome: Decisive British victory, marked a major turning point in the North African Campaign and led to the defeat of Axis forces in Africa
	+ Significance: First major victory for the Allies in North Africa, and set the stage for the eventual invasion of Italy and the Mediterranean region 

**<font color='magenta'>Total time:</font>** 293.83 sec.

## Multiple steps reasoning (task chain)

In [31]:
response = query_model(
    system_message,
    user_message.format(
    adjective="most important",
    number="5",
    items="battles",
    place="WW2"
    ), 
    max_length=512)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** 
What are the most important 5 battles from WW2?
 

**<font color='green'>Answer:</font>** Here are the top 5 most important battles of World War 2, in no particular order:

**The Top 5 Most Important Battles of World War 2:**

* **Battle of Stalingrad (August 1942 - February 1943)**:
	+ A major turning point on the Eastern Front, where the Soviet Union defeated Nazi Germany, marking a significant shift in the war's momentum.
	+ Resulted in the loss of over 300,000 German soldiers and a major blow to Hitler's military strategy.
* **Battle of Midway (June 4-7, 1942)**:
	+ A naval battle in the Pacific that prevented a Japanese invasion of Hawaii and turned the tide of the war in the Pacific.
	+ The Japanese lost four aircraft carriers, a significant blow to their naval power, while the US lost one aircraft carrier and a destroyer.
* **Battle of El Alamein (October-November 1942)**:
	+ A decisive British victory in North Africa that halted the Axis advance and led to the eventual defeat of German and Italian forces in Africa.
	+ Marked a major shift in the balance of power in the Mediterranean and set the stage for the Allied invasion of Italy.
* **Battle of Normandy (D-Day, June 6, 1944)**:
	+ The largest seaborne invasion in history, where Allied forces landed in Normandy, France, marking a crucial turning point in the war.
	+ Led to the liberation of Western Europe and the beginning of the end of Germany's control over the continent.
* **Battle of Berlin (April-May 1945)**:
	+ The final major battle of the war in Europe, where Soviet forces captured the German capital and led to the surrender of Germany.
	+ Marked the end of World War 2 in Europe and led to the division of Germany and the beginning of the Cold War.

These battles had a significant impact on the outcome of the war and the course of history. 

**<font color='magenta'>Total time:</font>** 249.49 sec.

## Reasoning like Einstein will do

In [32]:
system_message = """
You are a math professor, smart but cool.
Background: A train traveling from Bucharest to Ploiesti (60 km distance) has the speed of 60 km/h. 
The train starts in Bucharest and travels until Ploiesti, once, only in this direction.
A swallow, flying with 90 km/h, fly from Ploiesti to the moving train.
When it reaches the train, the swallow flies back toward Ploiesti,
ahead of the train. At Ploiesti turns again back and continues to fly back and forth 
(between the train approaching Ploiesti and Ploiesti) until the train reaches Ploiesti. 
The swallow will fly continously all the time the train is traveling from Bucharest to Ploiesti.
Reasoning: Think step by step. Explain your reasoning.
Question: {question}
Answer:
"""
response = query_model(
    system_message,
    user_message="How many kilometers will travel totally the swallow?", 
    temperature=0.1,
    max_length=512)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** How many kilometers will travel totally the swallow? 

**<font color='green'>Answer:</font>** My friend, let's break this down step by step.

First, let's calculate the time it takes for the train to travel from Bucharest to Ploiesti. The distance is 60 km, and the train's speed is 60 km/h, so the time it takes is:

Time = Distance / Speed = 60 km / 60 km/h = 1 hour

Now, let's consider the swallow's flight. When the swallow reaches the train, it flies back towards Ploiesti, which means it's flying at a speed of 90 km/h relative to the train. Since the train is moving at 60 km/h, the effective speed of the swallow relative to the ground is:

Effective speed = 90 km/h (swallow's speed) + 60 km/h (train's speed) = 150 km/h

The distance between the train and Ploiesti is 60 km, so the time it takes for the swallow to fly from the train to Ploiesti and back is:

Time = Distance / Effective speed = 60 km / 150 km/h = 0.4 hours

Since the swallow flies back and forth, it makes two trips for every hour the train travels. In 1 hour, the train travels 60 km, so the swallow makes 2 trips in that hour. In 0.4 hours, the swallow makes:

Number of trips = 0.4 hours x 2 trips/hour = 0.8 trips

The distance traveled by the swallow in one trip is the same as the distance between the train and Ploiesti, which is 60 km. Since the swallow makes 0.8 trips, the total distance traveled by the swallow is:

Total distance = Number of trips x Distance per trip = 0.8 trips x 60 km/trip = 48 km

However, this is not the total distance traveled by the swallow. The swallow also travels the distance between the train and Ploiesti in the last trip, which is 60 km. So, the total distance traveled by the swallow is:

Total distance = 48 km + 60 km = 108 km

Therefore, the swallow will travel a total of 108 kilometers. 

**<font color='magenta'>Total time:</font>** 283.19 sec.